In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [ ]:
tf.executing_eagerly()

In [ ]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
a = tf.zeros([], tf.float32)

from tensorflow.keras import layers, models

## Imports libs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from IPython.display import clear_output
%load_ext tensorboard

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
!pwd

In [ ]:
path1 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

In [ ]:
path3 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/Shap/CT_COVID/'
prefix3 = 'ct_covid'
path4 = '/home/maxwellsam/Multimodal-COVID19-Images/dataset/Shap/CT_NonCOVID/'
prefix4 = 'ct_noncovid'

In [ ]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

In [ ]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covid-19 negative

In [ ]:
ct_covid_features_df_shap =  processImages(path3,1)#1--> covid-19 positive
ct_noncovid_features_df_shap =  processImages(path4,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

In [ ]:
cvd_imgs_shap = [ct_noncovid_features_df_shap, ct_covid_features_df_shap]
cvd_imgs_dataset_shap = pd.concat(cvd_imgs_shap)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap = cvd_imgs_dataset_shap.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap)

#### Prepare negative covid images for machine learning ready

In [ ]:
input_data_x_ = ct_noncovid_features_df.iloc[:,:-1].to_numpy().reshape((2173,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df['class_label'])
ct_noncovid_features_df
ct_noncovid_features_df = pd.get_dummies(ct_noncovid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_ = np.array(ct_noncovid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_negative_only = input_data_x_
output_label_y_negative_only = output_label_y_
print('Input_x Data Shape: \n{0}'.format(input_data_x_negative_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_negative_only.shape))

#### Prepare positive covid images for machine learning ready

In [ ]:
input_data_x__ = ct_covid_features_df.iloc[:,:-1].to_numpy().reshape((2476,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df['output_encode'] = label_encoder.fit_transform(ct_covid_features_df['class_label'])
ct_covid_features_df
ct_covid_features_df = pd.get_dummies(ct_covid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__ = np.array(ct_covid_features_df[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_positive_only = input_data_x__
output_label_y_positive_only = output_label_y__
print('Input_x Data Shape: \n{0}'.format(input_data_x_positive_only.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_positive_only.shape))

#### Prepare both positive and negetive covid images for machine learning ready

In [ ]:
input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((4649,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))

In [ ]:
cvd_imgs_dataset

In [ ]:
ct_covid_features_df_shap.shape

#### Prepare negative covid shap-images for machine learning ready

In [ ]:
input_data_x_shap = ct_noncovid_features_df_shap.iloc[:,:-1].to_numpy().reshape((1738,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df_shap['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df_shap['class_label'])
ct_noncovid_features_df_shap
ct_noncovid_features_df_shap = pd.get_dummies(ct_noncovid_features_df_shap, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_shap = np.array(ct_noncovid_features_df_shap[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_negative_only_shap = input_data_x_shap
output_label_y_negative_only_shap = output_label_y_shap
print('Input_x Data Shape: \n{0}'.format(input_data_x_negative_only_shap.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_negative_only_shap.shape))

In [ ]:
## Images of Negative Covid-19
for i in range(4):
    imshow(input_data_x_negative_only_shap[i])
    show()

#### Prepare positive covid shap-images for machine learning ready

In [ ]:
input_data_x__shap = ct_covid_features_df_shap.iloc[:,:-1].to_numpy().reshape((1980,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df_shap['output_encode'] = label_encoder.fit_transform(ct_covid_features_df_shap['class_label'])
ct_covid_features_df_shap
ct_covid_features_df_shap = pd.get_dummies(ct_covid_features_df_shap, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__shap = np.array(ct_covid_features_df_shap[['class_label']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
input_data_x_positive_only_shap = input_data_x__shap
output_label_y_positive_only_shap = output_label_y__shap
print('Input_x Data Shape: \n{0}'.format(input_data_x_positive_only_shap.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_positive_only_shap.shape))

In [ ]:
## Images of Negative Covid-19
for i in range(4):
    imshow(input_data_x_positive_only_shap[i])
    show()

#### Prepare both positive and negetive covid shap images for machine learning ready

In [ ]:
input_data_x_shap = cvd_imgs_dataset_shap.iloc[:,:-1].to_numpy().reshape((3718,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap['class_label'])
cvd_imgs_dataset_shap
cvd_imgs_dataset_shap = pd.get_dummies(cvd_imgs_dataset_shap, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_shap = np.array(cvd_imgs_dataset_shap[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x_shap.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_shap.shape))

In [ ]:
cvd_imgs_dataset_shap

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    input_data_x, output_label_y, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap, test_features_shap, train_labels_shap, test_labels_shap = train_test_split(
    input_data_x_shap, output_label_y_shap, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_negative_only, test_features_negative_only, train_labels_negative_only, test_labels_negative_only = train_test_split(
    input_data_x_negative_only, output_label_y_negative_only, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_positive_only, test_features_positive_only, train_labels_positive_only, test_labels__positive_only = train_test_split(
    input_data_x_positive_only, output_label_y_positive_only, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_negative_only_shap, test_features_negative_only_shap, train_labels_negative_only_shap, test_labels_negative_only_shap = train_test_split(
    input_data_x_negative_only_shap, output_label_y_negative_only_shap, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_positive_only_shap, test_features_positive_only_shap, train_labels_positive_only_shap, test_labels__positive_only_shap = train_test_split(
    input_data_x_positive_only_shap, output_label_y_positive_only_shap, test_size=.20, random_state=42)

In [ ]:
train_features_shap.shape

In [ ]:
train_labels

In [ ]:
print(type(train_features_positive_only[0]))

### Loading the dataset

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        """
        Args:
            features (array-like): Input image features (e.g., images as numpy arrays).
            labels (array-like): Corresponding labels for the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        image = self.features[idx]
        label = self.labels[idx]

        # Ensure the image has the correct shape
        if image.ndim == 2:  # Grayscale image (height, width)
            image = np.expand_dims(image, axis=-1)  # Convert to (height, width, 1)
        elif image.ndim == 3 and image.shape[2] == 1:  # If the image is (height, width, 1)
            image = np.squeeze(image, axis=-1)  # Convert to (height, width)

        # Convert to PIL image
        if isinstance(image, np.ndarray):
            if image.ndim == 2:  # Grayscale
                image = Image.fromarray(image)  # Convert from numpy array to PIL Image
            elif image.ndim == 3:  # RGB
                image = Image.fromarray(image.astype('uint8'), 'RGB')

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from torchvision import transforms

# Custom normalization for grayscale images (mean and std for single channel)
# normalize_grayscale = transforms.Normalize(mean=[0.485], std=[0.229])

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     normalize_grayscale,  # Apply normalization only for grayscale images
# ])

# Use a conditional normalization based on the number of channels
def dynamic_normalize(tensor):
    """
    Normalize tensors dynamically based on the number of channels.
    Args:
        tensor: A PyTorch tensor representing the image.
    Returns:
        Normalized tensor.
    """
    if tensor.shape[0] == 3:  # RGB Image
        return transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(tensor)
    elif tensor.shape[0] == 1:  # Grayscale Image
        return transforms.Normalize(mean=[0.485], std=[0.229])(tensor)
    else:
        raise ValueError(f"Unsupported number of channels: {tensor.shape[0]}")

# Add this dynamic normalization into your transform chain:
transform = transforms.Compose([
    transforms.ToTensor(),
    dynamic_normalize,  # Apply normalization dynamically based on channels
])

In [ ]:
# train_dataset = CustomImageDataset(features=train_features, labels=train_labels, transform=transform)
# #test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# # Create DataLoader instances
# loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# #test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


### Load datset for NonCOVID

In [ ]:
train_dataset_negative = CustomImageDataset(features=train_features_negative_only, labels=train_labels_negative_only, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_negative = DataLoader(train_dataset_negative, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
train_dataset_negative_shap = CustomImageDataset(features=train_features_negative_only_shap, labels=train_labels_negative_only_shap, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_negative_shap = DataLoader(train_dataset_negative_shap, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


### Load datset for COVID

In [ ]:
train_dataset_positive = CustomImageDataset(features=train_features_positive_only, labels=train_labels_positive_only, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_positive = DataLoader(train_dataset_positive, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
train_dataset_positive_shap = CustomImageDataset(features=train_features_positive_only_shap, labels=train_labels_positive_only_shap, transform=transform)
#test_dataset = CustomImageDataset(features=test_features, labels=test_labels, transform=transform)

# Create DataLoader instances
loader_positive_shap = DataLoader(train_dataset_positive_shap, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
loader_positive_shap                                       

### Building the multimodal GAN-Transfomer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import torch.nn.functional as F


class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128 * 8 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Unflatten(1, (128, 8, 8)),
            nn.ConvTranspose2d(128, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(128, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(128, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # Example Convolutional Layers
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # Additional layers
            nn.Flatten(), 
            nn.Linear(64 * 64 * 64, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        print("This is: ")
        print(x.shape)
        x = self.model(x)
        return self.sigmoid(x)


class GAN(nn.Module):
    def __init__(self, generator, discriminator):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
    
    def forward(self, x):
        generated_image = self.generator(x)
        gan_output = self.discriminator(generated_image)
        return gan_output

# Vision Transformer Model
def VisionTransformer(input_shape=(300, 300, 1)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same')(inputs)  
    x = layers.Flatten()(x)
    x = layers.Dense(2 * 2 * 32, activation='relu')(x)
    
    # Reshaping to scalar output (remove the Conv2DTranspose layers)
    x = layers.Dense(1, activation='linear')(x)  # Output a scalar for each image

    model = Model(inputs, x)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])
    return model


# Save generated images
def save_generated_images(epoch, generator, latent_dim, save_dir='gan_generated_images'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    noise = np.random.randn(100, latent_dim)
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2.0
    for i in range(10):
        tf.keras.preprocessing.image.save_img(os.path.join(save_dir, f'gen_img_{epoch}_{i}.png'), generated_images[i])


In [ ]:
# latent_dim = 100
# generator = Generator(latent_dim)
# discriminator = Discriminator()
# device = "cuda" if torch.cuda.is_available() else "cpu"
# generator.to(device)
# discriminator.to(device)
# gan = GAN(generator, discriminator)

# # Vision Transformer training setup
# vit_model = VisionTransformer(input_shape=(300, 300, 1))
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # Train GAN
# def train_gan(generator, discriminator, gan, dataloader, latent_dim, n_epochs, n_batch):
#     # Define optimizer
#     discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
#     generator_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

#     criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy loss

#     for epoch in range(n_epochs):
#         for batch in dataloader:
#             # Get real images
            
#             real_images = batch[0].to(device)
#             # Generate fake images
#             noise = torch.randn(n_batch, latent_dim).to(device)
#             fake_images = generator(noise)

#             # Labels for real and fake images
#             real_labels = torch.ones(n_batch, 1).to(device)
#             fake_labels = torch.zeros(n_batch, 1).to(device)
#             fake_images_resized = F.interpolate(fake_images, size=(300, 300), mode='bilinear', align_corners=False)
            
            
#             # Train the discriminator
#             discriminator_optimizer.zero_grad()

#             # Concatenate real and fake images
#             d_input = torch.cat((real_images, fake_images_resized))
#             d_labels = torch.cat((real_labels, fake_labels))

#             # Get the discriminator output (shape: [n_batch * 2, 1])
#             d_output = discriminator(d_input)
            
#             # Compute the discriminator loss
#             d_labels_ = d_labels[:d_output.size(0)]
#             d_loss = criterion(d_output, d_labels_)
#             d_loss.backward()

#             # Update discriminator
#             discriminator_optimizer.step()

#             # Train the generator
#             generator_optimizer.zero_grad()

#             # Generate new noise for the generator
#             noise = torch.randn(n_batch, latent_dim).to(device)

#             # Get the generator's output for the fake images
#             g_output = generator(noise)
#             g_output_resized = F.interpolate(g_output, size=(300, 300), mode='bilinear', align_corners=False)

#             # Labels for the generator are all real (1) since we want the generator to fool the discriminator
#             g_labels = torch.ones(n_batch, 1).to(device)

#             # Get the discriminator's output for the generator's fake images
#             g_d_output = discriminator(g_output_resized)

#             # Compute the generator loss
#             g_loss = criterion(g_d_output, g_labels)
#             g_loss.backward()

#             # Update generator
#             generator_optimizer.step()

#         print(f"Epoch {epoch+1}/{n_epochs} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")


# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import smart_resize

# def preprocess_images(images):
#     images_resized = tf.image.resize(images, (30, 30))
#     return images_resized


# # Train Vision Transformer
# def train_vit(vit_model, dataloader, n_epochs=10, n_batch=32):
#     for epoch in range(n_epochs):
#         for batch in dataloader:
#             images, labels = batch
#             # Ensure images and labels have matching batch sizes
#             if images.size(0) == labels.size(0):
#                 images = tf.convert_to_tensor(images.cpu().numpy().reshape(-1, 300, 300, 1))
#                 labels = tf.convert_to_tensor(labels.cpu().numpy())
#                 vit_model.fit(images, labels)
#             else:
#                 print(f"Batch size mismatch: {images.size(0)} images, {labels.size(0)} labels")




# # Training process
# train_gan(generator, discriminator, gan, loader_positive, latent_dim, n_epochs=10, n_batch=64)
# train_vit(vit_model, loader_positive_shap, n_epochs=10, n_batch=192)

# # Save ViT output as images
# def save_vit_output(vit_model, dataset, save_dir='/home/maxwellsam/Multimodal-COVID19-Images/generatedImages/Transformer/COVID-19/'):
#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir)
#     predictions = vit_model.predict(dataset)
#     for i in range(10):
#         img = tf.image.resize(predictions[i], (30, 30))
#         tf.keras.preprocessing.image.save_img(os.path.join(save_dir, f'vit_img_{i}.png'), img)

# save_vit_output(vit_model, train_dataset_positive_shap[:10])

In [ ]:
class Discriminator_(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator_, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_shape[0], 64, kernel_size=4, stride=2, padding=1),  # Conv layer 1
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # Conv layer 2
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # Conv layer 3
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),  # Conv layer 4
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),  # Flatten the tensor to pass it to the fully connected layers
            nn.Linear(512 * 8 * 8, 1),  # Fully connected layer
            nn.Sigmoid()  # Output a probability
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
latent_dim = 100
generator = Generator(latent_dim)
discriminator = Discriminator()

In [ ]:
# input_tensor = torch.randn(64, 3, 64, 64)  
# output = discriminator(input_tensor)

In [ ]:
gan = GAN(generator, discriminator)
vit_model = VisionTransformer(input_shape=(300, 300, 1))

In [ ]:
import os
import torch
from torch.utils.tensorboard import SummaryWriter
import torchvision

def train_multimodal_gan(generator, discriminator, gan, vit_model, loader_gan, loader_vit, latent_dim, num_epochs=10, n_batch=64, save_dir='generated_images/', device='cuda'):
    """
    Trains the GAN and ViT models on separate datasets, saves generated images to a directory,
    and logs to TensorBoard.
    
    Parameters:
    - generator: GAN generator model.
    - discriminator: GAN discriminator model.
    - gan: The complete GAN model.
    - vit_model: Pre-trained Vision Transformer model.
    - loader_gan: Dataset for GAN training.
    - loader_vit: Dataset for ViT training.
    - latent_dim: Latent vector dimension for GAN.
    - num_epochs: Number of epochs to train the model.
    - n_batch: Batch size.
    - save_dir: Directory to save the images.
    - device: Device to use for training (e.g., 'cuda' or 'cpu').
    """
    # TensorBoard writers
    writer_fake = SummaryWriter('runs/GANViT_MNIST/fake')
    writer_real = SummaryWriter('runs/GANViT_MNIST/real')
    writer_losses = SummaryWriter('runs/GANViT_MNIST/losses')

    # Start TensorBoard
    %tensorboard --logdir=runs/CNN_GAN_MNIST/

    # Create the directory if it does not exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    step = 0
    fixed_noise = torch.randn(n_batch, latent_dim, device=device) 

    for epoch in range(num_epochs):
        for batch_idx, (real, _) in enumerate(loader_gan):  
            real = real.to(device)
            batch_size = real.shape[0]

            # Train Discriminator
            noise = torch.randn(batch_size, latent_dim).to(device)
            fake = generator(noise)
            print(fake.shape)
            
            disc_real = discriminator(real).view(-1)
            lossD_real = torch.nn.functional.binary_cross_entropy(disc_real, torch.ones_like(disc_real))
            
            disc_fake = discriminator(fake.detach()).view(-1)
            lossD_fake = torch.nn.functional.binary_cross_entropy(disc_fake, torch.zeros_like(disc_fake))
            lossD = (lossD_real + lossD_fake) / 2
            discriminator.zero_grad()
            lossD.backward()
            discriminator.optim.step()

            # Train Generator
            output = discriminator(fake).view(-1)
            lossG = torch.nn.functional.binary_cross_entropy(output, torch.ones_like(output))
            generator.zero_grad()
            lossG.backward()
            generator.optim.step()

            # Print loss every 100 batches
            if batch_idx % 100 == 0:
                print(f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader_gan)} "
                      f"Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")

                with torch.no_grad():
                    fake_images = generator(fixed_noise)
                    img_grid_fake = torchvision.utils.make_grid(fake_images, normalize=True)
                    img_grid_real = torchvision.utils.make_grid(real, normalize=True)

                    # Log images to TensorBoard
                    writer_fake.add_image("Fake Images", img_grid_fake, global_step=step)
                    writer_real.add_image("Real Images", img_grid_real, global_step=step)
                    writer_losses.add_scalar("Discriminator Loss", lossD, global_step=step)
                    writer_losses.add_scalar("Generator Loss", lossG, global_step=step)

                    # Save generated images to disk
                    save_generated_images(fake_images, real, epoch, batch_idx, save_dir)

                step += 1

        # After each epoch, save generated images to a directory
        save_generated_images(fake_images, real, epoch, batch_idx, save_dir)

    writer_fake.close()
    writer_real.close()
    writer_losses.close()


def save_generated_images(fake_images, real_images, epoch, batch_idx, save_dir):
    """
    Saves the generated images after each batch or epoch to the specified directory.
    
    Parameters:
    - fake_images: Generated images by the GAN model.
    - real_images: Real images from the dataset.
    - epoch: Current epoch number.
    - batch_idx: Current batch index.
    - save_dir: Directory to save the images.
    """
    # Save the real and fake images to disk
    for i, img in enumerate(fake_images):
        img_path = os.path.join(save_dir, f'fake_epoch_{epoch+1}_batch_{batch_idx+1}_img_{i+1}.png')
        torchvision.utils.save_image(img, img_path)

    for i, img in enumerate(real_images):
        img_path = os.path.join(save_dir, f'real_epoch_{epoch+1}_batch_{batch_idx+1}_img_{i+1}.png')
        torchvision.utils.save_image(img, img_path)


In [ ]:
train_multimodal_gan(generator, discriminator, gan, vit_model, loader_positive, loader_positive_shap, latent_dim=100, num_epochs=10, n_batch=64)
